In [ ]:
def Tuning(user_email):
    optimize_ROI_df = pd.DataFrame(columns = ['strategy', 'stock', 'x', 'y', 'z'])
    best_ROI_df = pd.DataFrame(columns = ['strategy', 'stock', 'x', 'y', 'z'])
    z_name = "Return"
    for _, user_stock_row in user_stock_df[user_stock_df.email == user_email].iterrows():
        strategy_df=get_strategy_df(str(user_stock_row.strategy)).copy()
        for _, strategy_row in strategy_df.iterrows():
            x_name=str(strategy_row.x_name)
            y_name=str(strategy_row.y_name)
            x_value=int(strategy_row.x_value)
            y_value=int(strategy_row.y_value)
            x0=int(strategy_row.x0)
            x_max=int(strategy_row.x_max)
            dx=int(strategy_row.dx)
            y0=int(strategy_row.y0)
            y_max=int(strategy_row.y_max)
            dy=int(strategy_row.dy)
        column_names = [x_name, y_name, z_name]
        ROI_df = pd.DataFrame(columns = column_names)
        max_x = 0
        max_y = 0
        max_ROI = -100
        print(user_stock_row.stock)
        for x in range(x0, x_max+1, dx):
            for y in range(y0, y_max+1, dy):
                position = backtesting(user_stock_row, str(x), str(y))
                ROI = ((position.value[-1]-position.value[0])/position.value[0]).round(2)
                new_row = {x_name:x, y_name:y, z_name:ROI}
                ROI_df = ROI_df.append(new_row, ignore_index=True)
                if ROI>max_ROI:
                    max_ROI = ROI
                    max_x = x
                    max_y = y
                new_optimize_row = {'strategy':str(user_stock_row.strategy), 'stock':str(user_stock_row.stock), 'x':x, 'y':y, 'z':ROI}
                optimize_ROI_df = optimize_ROI_df.append(new_optimize_row, ignore_index=True)
        plot_ROI_contour(user_stock_row, ROI_df, strategy_df)
        new_best_row = {'strategy':str(user_stock_row.strategy), 'stock':str(user_stock_row.stock), 'x':max_x, 'y':max_y, 'z':max_ROI}
        best_ROI_df = best_ROI_df.append(new_best_row, ignore_index=True)
        print(str(user_stock_row.strategy) + '(' + str(max_x) + ', ' + str(max_y) + ') is better for ' + str(user_stock_row.stock))
        plot_ROI_contour(user_stock_row, ROI_df, strategy_df, relative=True)
    return best_ROI_df, optimize_ROI_df